In [30]:
from owslib.wms import WebMapService

# hel_map = WebMapService('https://kartta.hel.fi/ws/geoserver/avoindata/wms?request=getCapabilities')
hel_map = WebMapService(
    'https://kartta.luke.fi/geoserver/MVMI/wms?version=1.3.0')


In [35]:
print(f'Title: {hel_map.identification.title}\nVersion: {hel_map.identification.version}\nAbstract: {hel_map.identification.abstract}')
layer_list = [
    name for name in hel_map.contents
    if ('kuusi' in name)
]


Title: GeoServer Web Map Service
Version: 1.1.1
Abstract: A compliant implementation of WMS plus most of the SLD extension (dynamic styling). Can also generate PDF, SVG, KML, GeoRSS


In [70]:
info_list = []
for layer in layer_list:
    print(
        f' - Title: {hel_map[layer].title}\n - Range: {hel_map[layer].boundingBoxWGS84}\n - crsOptions: {hel_map[layer].crsOptions}\n - Styles: {hel_map[layer].styles}\n'
    )
    info_list.append({'layer':layer, 'box': hel_map[layer].boundingBoxWGS84, 'crs':hel_map[layer].crsOptions, 'style':list(hel_map[layer].styles.keys())})


 - Title: Biomassa, kuusi, elävät oksat 2009 (10 kg/ha)
 - Range: (15.604854897351455, 59.35183067646301, 33.12630922296247, 70.07679148715648)
 - crsOptions: ['EPSG:4326', 'EPSG:3048', 'EPSG:3047', 'EPSG:3046', 'EPSG:3034', 'EPSG:3067', 'EPSG:4258', 'EPSG:3035']
 - Styles: {'bm_kuusi_elavatoksat_0610': {'title': 'Biomassa, kuusi, elävät oksat (10 kg/ha)', 'legend': 'https://kartta.luke.fi/geoserver/MVMI/wms?request=GetLegendGraphic&version=1.1.1&format=image%2Fpng&width=20&height=20&layer=bm_kuusi_elavatoksat_0610'}}

 - Title: Biomassa, kuusi, elävät oksat 2011 (10 kg/ha)
 - Range: (15.604854897351455, 59.35183067646301, 33.12630922296247, 70.07679148715648)
 - crsOptions: ['EPSG:4326', 'EPSG:3048', 'EPSG:3047', 'EPSG:3046', 'EPSG:3034', 'EPSG:3067', 'EPSG:4258', 'EPSG:3035']
 - Styles: {'bm_kuusi_elavatoksat_0610': {'title': 'Biomassa, kuusi, elävät oksat (10 kg/ha)', 'legend': 'https://kartta.luke.fi/geoserver/MVMI/wms?request=GetLegendGraphic&version=1.1.1&format=image%2Fpng&width

In [38]:
# [op.name for op in hel_map.operations]

['GetCapabilities',
 'GetMap',
 'GetFeatureInfo',
 'DescribeLayer',
 'GetLegendGraphic',
 'GetStyles']

In [39]:
# hel_map.getOperationByName('GetMap').methods

[{'type': 'Get',
  'url': 'https://kartta.luke.fi/geoserver/MVMI/wms?SERVICE=WMS&'}]

In [40]:
# hel_map.getOperationByName('GetMap').formatOptions

['image/png',
 'application/atom xml',
 'application/atom+xml',
 'application/json;type=utfgrid',
 'application/openlayers',
 'application/openlayers2',
 'application/openlayers3',
 'application/pdf',
 'application/rss xml',
 'application/rss+xml',
 'application/vnd.google-earth.kml',
 'application/vnd.google-earth.kml xml',
 'application/vnd.google-earth.kml+xml',
 'application/vnd.google-earth.kml+xml;mode=networklink',
 'application/vnd.google-earth.kmz',
 'application/vnd.google-earth.kmz xml',
 'application/vnd.google-earth.kmz+xml',
 'application/vnd.google-earth.kmz;mode=networklink',
 'atom',
 'image/geotiff',
 'image/geotiff8',
 'image/gif',
 'image/gif;subtype=animated',
 'image/jpeg',
 'image/png8',
 'image/png; mode=8bit',
 'image/svg',
 'image/svg xml',
 'image/svg+xml',
 'image/tiff',
 'image/tiff8',
 'image/vnd.jpeg-png',
 'image/vnd.jpeg-png8',
 'kml',
 'kmz',
 'openlayers',
 'rss',
 'text/html; subtype=openlayers',
 'text/html; subtype=openlayers2',
 'text/html; subtyp

In [65]:
for info in info_list:
    img = hel_map.getmap(layers=[info['layer']],
                         styles=['kuusitukki_0610'],
                         srs=info['crs'][0],
                         bbox=(15.604854897351455, 59.35183067646301,
                               33.12630922296247, 65.07679148715648),
                         size=(512, 512),
                         format='image/png',
                         transparent=True)
    with open(f'./data/{layer}.png', 'wb') as f:
        f.write(img.read())
    break


In [71]:
hel_map[layer].styles

{'kuusitukki_0610': {'title': 'Kuusi, tukkipuu (m3/ha)',
  'legend': 'https://kartta.luke.fi/geoserver/MVMI/wms?request=GetLegendGraphic&version=1.1.1&format=image%2Fpng&width=20&height=20&layer=kuusitukki_1519'}}

In [73]:
info_list[0]['style']

['bm_kuusi_elavatoksat_0610']